# Exploratory Data Analysis

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import streamlit as st


In [5]:
df = pd.read_csv('processed_donner_pass.csv')

In [7]:
pivot_by_month = df.pivot_table(index='year_month')
print(pivot_by_month)

            air_temp_max_f  air_temp_min_f  daily_precip_cm  month  \
year_month                                                           
1970-10          55.718194       31.677742         0.258097     10   
1970-11          42.689300       28.303700         1.476587     11   
1970-12          31.033226       16.306032         1.426497     12   
1971-01          38.412935       17.788419         0.667774      1   
1971-02          39.670893       16.408464         0.231321      2   
...                    ...             ...              ...    ...   
2019-05          53.019355       30.258065         0.564516      5   
2019-06          67.700000       38.540000         0.023333      6   
2019-07          73.283871       44.832258         0.000000      7   
2019-08          75.432258       46.283871         0.000000      8   
2019-09          63.560000       39.260000         0.206667      9   

            new_snow_cm  season_total_precip_cm  season_total_snow_cm  \
year_month      

C:\Users\jncol\AppData\Local\Temp\ipykernel_9900\3571148933.py:1: FutureWarning:

pivot_table dropped a column because it failed to aggregate. This behavior is deprecated and will raise in a future version of pandas. Select only the columns that can be aggregated.

